# FastAPI with Celery and Redis

In [2]:
import requests

In [26]:
response = requests.get('http://localhost:8000/add')
response = response.json()
task_id = response['task_id']
task_id

'e4652de6-4d85-402e-9063-5fa47d25f986'

In [30]:
response = requests.get(f'http://localhost:8000/task/status/{task_id}')
print(response.json())

{'task_status': 'SUCCESS'}


In [31]:
response = requests.get(f'http://localhost:8000/task/result/{task_id}')
print(response.json())

{'task_result': 12}


In [32]:
response = requests.get('http://localhost:8000/group/add')
response = response.json()
task_id = response['task_id']
task_id

'91a17921-26b9-47a2-8ccc-bc0322dcacd9'

In [36]:
response = requests.get(f'http://localhost:8000/group/status/{task_id}')
print(response.json())

{'successful': True}


In [37]:
response = requests.get(f'http://localhost:8000/group/result/{task_id}')
print(response.json())

{'group_result': [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38]}


# Celery Group

In [38]:
from worker import celery_app, add
from celery.result import AsyncResult, GroupResult
from celery import group, chord, chain

In [39]:
jobs = group(add.s(i, i) for i in range(2))
result = jobs.apply_async()
result.save()
group_id = result.id

In [40]:
# results = GroupResult.restore(task_id, app=celery_app)
restored = GroupResult.restore(group_id, app=celery_app)

if restored.ready():
    print("All subtasks completed.")

In [43]:
restored.successful()

True

In [44]:
restored.get()

[0, 2]